In [244]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [245]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
import tensorflow as tf
import gradio as gr

In [246]:
leafly = pd.read_csv('C:/Users/juhyn/OneDrive/Desktop/group-project-3/data/leafly_strain_data.csv')
leafly.head()

,name,img_url,type,thc_level,most_common_terpene,description,relaxed,happy,euphoric,uplifted,...,fibromyalgia,crohn's_disease,phantom_limb_pain,epilepsy,multiple_sclerosis,parkinson's,tourette's_syndrome,alzheimer's,hiv/aids,tinnitus
0,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
1,Wedding Cake,https://images.leafly.com/flower-images/weddin...,Hybrid,22%,Limonene,Wedding Cake is a potent indica-hybrid marijua...,60%,50%,41%,35%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
2,Runtz,https://images.leafly.com/flower-images/runtz-...,Hybrid,19%,Limonene,Runtz is a rare hybrid marijuana strain by Coo...,47%,62%,48%,43%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
3,Gelato,https://images.leafly.com/flower-images/gelato...,Hybrid,17%,Caryophyllene,"Gelato, aka Larry Bird, is a hybrid marijuana ...",54%,49%,46%,37%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
4,Blue Dream,https://images.leafly.com/flower-images/blue-d...,Hybrid,18%,Myrcene,Blue Dream is a sativa-dominant hybrid marijua...,48%,57%,52%,48%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%


In [247]:
leafly.columns

Index(['name', 'img_url', 'type', 'thc_level', 'most_common_terpene',
       'description', 'relaxed', 'happy', 'euphoric', 'uplifted', 'sleepy',
       'dry_mouth', 'dry_eyes', 'dizzy', 'paranoid', 'anxious', 'stress',
       'pain', 'depression', 'anxiety', 'insomnia', 'hungry', 'talkative',
       'headache', 'ptsd', 'creative', 'energetic', 'fatigue', 'focused',
       'giggly', 'lack_of_appetite', 'nausea', 'headaches', 'bipolar_disorder',
       'cancer', 'tingly', 'cramps', 'aroused', 'gastrointestinal_disorder',
       'inflammation', 'muscle_spasms', 'eye_pressure', 'migraines', 'asthma',
       'anorexia', 'arthritis', 'add/adhd', 'muscular_dystrophy',
       'hypertension', 'glaucoma', 'pms', 'seizures', 'spasticity',
       'spinal_cord_injury', 'fibromyalgia', 'crohn's_disease',
       'phantom_limb_pain', 'epilepsy', 'multiple_sclerosis', 'parkinson's',
       'tourette's_syndrome', 'alzheimer's', 'hiv/aids', 'tinnitus'],
      dtype='object')

In [248]:
lab_results = pd.read_csv('C:/Users/juhyn/OneDrive/Desktop/group-project-3/data/results.csv')
lab_results.head()

C:\Users\juhyn\AppData\Local\Temp\ipykernel_21148\1654712513.py:1: DtypeWarning: Columns (2,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_results = pd.read_csv('C:/Users/juhyn/OneDrive/Desktop/group-project-3/data/results.csv')


,Database Identifier,Database Name,Test Result UID,Sample Name,Sample Type,Receipt Time,Test Time,Post Time,Provider,cis-Nerolidol,...,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content
0,analytical360,Analytical 360,NaN,BHO Blackberry 22,Archived,NaN,NaN,2013-08-19,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.08,NaN,NaN,NaN
1,analytical360,Analytical 360,NaN,BHO The Sour Bud,Archived,NaN,NaN,2013-08-19,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.75,0.34,NaN,NaN
2,analytical360,Analytical 360,NaN,Northern Lights,Archived,NaN,NaN,2013-08-19,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.05,NaN,NaN
3,analytical360,Analytical 360,NaN,Blueberry Kush,Archived,NaN,NaN,2013-08-19,3.0,NaN,...,NaN,0.04,NaN,NaN,NaN,NaN,0.25,0.19,NaN,NaN
4,analytical360,Analytical 360,NaN,White Dawg,Archived,NaN,NaN,2013-08-19,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.12,0.11,NaN,NaN


In [249]:
lab_results.columns

Index(['Database Identifier', 'Database Name', 'Test Result UID',
       'Sample Name', 'Sample Type', 'Receipt Time', 'Test Time', 'Post Time',
       'Provider', 'cis-Nerolidol', 'trans-Nerolidol', 'trans-Nerolidol 1',
       'trans-Nerolidol 2', 'trans-Ocimene', '3-Carene', 'Camphene',
       'Caryophyllene Oxide', 'Eucalyptol', 'Geraniol', 'Guaiol', 'Isopulegol',
       'Linalool', 'Ocimene', 'Terpinolene', 'alpha-Bisabolol',
       'alpha-Humulene', 'alpha-Pinene', 'alpha-Terpinene',
       'beta-Caryophyllene', 'beta-Myrcene', 'beta-Ocimene', 'beta-Pinene',
       'delta-Limonene', 'gamma-Terpinene', 'p-Cymene', 'delta-9 THC-A',
       'delta-9 THC', 'delta-8 THC', 'THC-A', 'THCV', 'CBN', 'CBD-A', 'CBD',
       'CBDV', 'CBDV-A', 'delta-9 CBG-A', 'delta-9 CBG', 'CBC',
       'Moisture Content'],
      dtype='object')

In [250]:
# copying  lab_results dataframe
lab_results_copy = lab_results.copy()

# dropping columns from lab_results
lab_results_copy = lab_results_copy.drop(['Receipt Time', 'Test Time', 'Post Time', 'Provider'], axis = 1)

lab_results_copy

,Database Identifier,Database Name,Test Result UID,Sample Name,Sample Type,cis-Nerolidol,trans-Nerolidol,trans-Nerolidol 1,trans-Nerolidol 2,trans-Ocimene,...,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content
0,analytical360,Analytical 360,NaN,BHO Blackberry 22,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.08,NaN,NaN,NaN
1,analytical360,Analytical 360,NaN,BHO The Sour Bud,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.75,0.34,NaN,NaN
2,analytical360,Analytical 360,NaN,Northern Lights,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.05,NaN,NaN
3,analytical360,Analytical 360,NaN,Blueberry Kush,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,0.04,NaN,NaN,NaN,NaN,0.25,0.19,NaN,NaN
4,analytical360,Analytical 360,NaN,White Dawg,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.12,0.11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43013,sclabs,SC Labs,NaN,Dream Tip THCa Primary,"Concentrate, Inhalable",NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.0,0.0,0.32,0.00,0.00,NaN
43014,sclabs,SC Labs,NaN,Cornbread Shatter Primary,"Concentrate, Inhalable",NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.16,0.00,0.0,0.0,3.11,0.26,0.00,NaN
43015,sclabs,SC Labs,NaN,Sample 1,"Flower, Inhalable",NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.02,0.00,0.0,0.0,0.33,0.11,0.00,NaN
43016,sclabs,SC Labs,NaN,Sample 4,"Flower, Inhalable",NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.04,0.00,0.0,0.0,0.69,0.12,0.00,NaN


In [251]:
matching_values = leafly['name'][leafly['name'].isin(lab_results_copy['Sample Name'])]
print(matching_values)

0                GG4
1       Wedding Cake
3             Gelato
4         Blue Dream
5       Purple Punch
            ...     
4624        Good Day
4647      Stone Free
4668     Dirty Widow
4726        Deadwood
4740     Walrus Kush
Name: name, Length: 1199, dtype: object


In [252]:
merged_df = pd.merge(leafly, lab_results_copy, left_on = 'name', right_on = 'Sample Name', how = 'inner')

merged_df.head()

,name,img_url,type,thc_level,most_common_terpene,description,relaxed,happy,euphoric,uplifted,...,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content
0,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,NaN,NaN,0.08,0.07,NaN,NaN,0.36,NaN,0.02,NaN
1,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,NaN,NaN,0.17,0.26,NaN,NaN,0.91,NaN,0.05,NaN
2,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,NaN,0.04,0.12,0.40,NaN,NaN,1.49,NaN,0.42,NaN
3,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,NaN,NaN,0.04,0.04,NaN,NaN,0.32,NaN,NaN,NaN
4,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,0.01,0.01,0.01,0.01,NaN,NaN,0.46,0.07,0.04,10.19


In [253]:
merged_df = merged_df.drop(['img_url'], axis=1)
merged_df

,name,type,thc_level,most_common_terpene,description,relaxed,happy,euphoric,uplifted,sleepy,...,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content
0,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,NaN,NaN,0.08,0.07,NaN,NaN,0.36,NaN,0.02,NaN
1,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,NaN,NaN,0.17,0.26,NaN,NaN,0.91,NaN,0.05,NaN
2,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,NaN,0.04,0.12,0.40,NaN,NaN,1.49,NaN,0.42,NaN
3,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,NaN,NaN,0.04,0.04,NaN,NaN,0.32,NaN,NaN,NaN
4,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,0.01,0.01,0.01,0.01,NaN,NaN,0.46,0.07,0.04,10.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10207,Deadwood,Indica,NaN,NaN,Deadwood is potent indica cross between Bubba ...,80%,40%,0%,20%,40%,...,NaN,NaN,0.07,NaN,NaN,NaN,NaN,0.05,0.02,NaN
10208,Deadwood,Indica,NaN,NaN,Deadwood is potent indica cross between Bubba ...,80%,40%,0%,20%,40%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04,NaN,NaN
10209,Deadwood,Indica,NaN,NaN,Deadwood is potent indica cross between Bubba ...,80%,40%,0%,20%,40%,...,NaN,0.05,0.07,0.07,NaN,NaN,NaN,0.16,NaN,NaN
10210,Walrus Kush,Hybrid,NaN,NaN,"While normally difficult to ignore, this Walru...",0%,80%,0%,40%,0%,...,NaN,NaN,0.05,0.07,NaN,NaN,0.24,NaN,0.02,NaN


In [254]:
merged_df = merged_df.fillna('')
print(merged_df.isnull().sum())

name                   0
type                   0
thc_level              0
most_common_terpene    0
description            0
                      ..
CBDV-A                 0
delta-9 CBG-A          0
delta-9 CBG            0
CBC                    0
Moisture Content       0
Length: 108, dtype: int64


In [255]:
# setting up VADER 
#analyze user input and sentiment scores
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# initializing VADER
analyzer = SentimentIntensityAnalyzer()

In [256]:
# analyzing the sentiment in the column description

def analyze_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

# applying function to description column
merged_df['compound score'] = merged_df['description'].apply(lambda x: analyze_sentiment(x) if pd.notnull else 0)

merged_df[['name', 'description', 'compound score']]

,name,description,compound score
0,GG4,"Original Glue (GG4), developed by GG Strains, ...",0.9049
1,GG4,"Original Glue (GG4), developed by GG Strains, ...",0.9049
2,GG4,"Original Glue (GG4), developed by GG Strains, ...",0.9049
3,GG4,"Original Glue (GG4), developed by GG Strains, ...",0.9049
4,GG4,"Original Glue (GG4), developed by GG Strains, ...",0.9049
...,...,...,...
10207,Deadwood,Deadwood is potent indica cross between Bubba ...,0.4767
10208,Deadwood,Deadwood is potent indica cross between Bubba ...,0.4767
10209,Deadwood,Deadwood is potent indica cross between Bubba ...,0.4767
10210,Walrus Kush,"While normally difficult to ignore, this Walru...",-0.6147


In [257]:
# loading hugging face sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# encoding the description column
merged_df['embeddings'] = merged_df['description'].apply(lambda x: model.encode(x, convert_to_tensor=True))

merged_df[['description', 'embeddings']]

,description,embeddings
0,"Original Glue (GG4), developed by GG Strains, ...","[tensor(-0.0391), tensor(-0.0044), tensor(0.06..."
1,"Original Glue (GG4), developed by GG Strains, ...","[tensor(-0.0391), tensor(-0.0044), tensor(0.06..."
2,"Original Glue (GG4), developed by GG Strains, ...","[tensor(-0.0391), tensor(-0.0044), tensor(0.06..."
3,"Original Glue (GG4), developed by GG Strains, ...","[tensor(-0.0391), tensor(-0.0044), tensor(0.06..."
4,"Original Glue (GG4), developed by GG Strains, ...","[tensor(-0.0391), tensor(-0.0044), tensor(0.06..."
...,...,...
10207,Deadwood is potent indica cross between Bubba ...,"[tensor(-0.0336), tensor(-0.0823), tensor(-0.0..."
10208,Deadwood is potent indica cross between Bubba ...,"[tensor(-0.0336), tensor(-0.0823), tensor(-0.0..."
10209,Deadwood is potent indica cross between Bubba ...,"[tensor(-0.0336), tensor(-0.0823), tensor(-0.0..."
10210,"While normally difficult to ignore, this Walru...","[tensor(0.0483), tensor(-0.0488), tensor(0.036..."


In [258]:
merged_df.columns

Index(['name', 'type', 'thc_level', 'most_common_terpene', 'description',
       'relaxed', 'happy', 'euphoric', 'uplifted', 'sleepy',
       ...
       'CBD-A', 'CBD', 'CBDV', 'CBDV-A', 'delta-9 CBG-A', 'delta-9 CBG', 'CBC',
       'Moisture Content', 'compound score', 'embeddings'],
      dtype='object', length=110)

In [259]:
# adding Langchain to handle user input
def get_dynamic_input(user_input):
    prompt = ChatPromptTemplate.from_template(
        "User wants recommendation based on: {user_input}, reply with a suitable recommendation"
    )
    return prompt.format(user_input=user_input)

In [260]:
# converting percentage to floats before scaling
def clean_percentage_columns(df, effect_columns):
    for col in effect_columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace('%', '', regex=False).astype(float) / 100.0
        else:
            df[col] = df[col] / 100.0
            
    return df

effect_columns = ['relaxed', 'happy', 'euphoric', 'uplifted', 'sleepy', 'dizzy', 'paranoid' ]

merged_df = clean_percentage_columns(merged_df, effect_columns)

# check clean values
for col in effect_columns:
    print(f"{col}: {merged_df[col].unique()}")


relaxed: [0.66 0.6  0.54 0.48 0.7  0.75 0.55 0.52 0.   0.59 0.56 0.38 0.65 0.42
 0.37 0.61 0.69 0.57 0.47 0.67 0.35 0.64 0.74 0.73 0.68 0.44 0.39 0.63
 0.4  0.58 0.53 0.71 0.49 0.78 0.62 0.43 0.28 0.51 0.79 0.5  0.41 0.23
 0.82 0.72 0.45 0.8  0.46 0.33 0.2  0.86 0.76 0.77 0.83 0.84 0.36 0.34
 0.14 0.81 1.   0.9  0.12 0.87 0.3  0.32 0.92 0.22 0.31 0.85 0.91 0.89
 0.88 0.27 0.25]
happy: [0.54 0.5  0.49 0.57 0.47 0.6  0.56 0.39 0.44 0.51 0.4  0.46 0.48 0.65
 0.52 0.53 0.62 0.55 0.32 0.61 0.42 0.58 0.64 0.43 0.59 0.45 0.63 0.38
 0.66 0.41 0.33 0.77 0.34 0.71 0.26 0.36 0.22 0.2  0.68 0.   0.67 0.25
 0.74 0.35 0.73 0.14 0.69 1.   0.7  0.12 0.27 0.37 0.72 0.31 0.75 0.82
 0.28 0.91 0.3  0.8  0.29 0.23 0.76 0.88 0.83 0.85 0.92 0.9 ]
euphoric: [0.51 0.41 0.46 0.52 0.4  0.42 0.5  0.49 0.3  0.43 0.   0.53 0.44 0.48
 0.54 0.39 0.45 0.36 0.55 0.47 0.24 0.57 0.34 0.37 0.56 0.33 0.63 0.31
 0.38 0.27 0.28 0.58 0.22 0.23 0.65 0.59 0.32 0.6  0.35 0.21 0.64 0.18
 0.69 0.68 0.62 0.19 0.9  0.25 0.76 0.66 0.

In [261]:
scaler = MinMaxScaler()
merged_df[effect_columns] = scaler.fit_transform(merged_df[effect_columns])

# recommendation function based on user input and description column
def recommend_terpene(user_input, top_n=5):
    user_embedding = model.encode(user_input, convert_to_tensor=True) # user input embedding

    similarities = np.inner(user_embedding, np.vstack(merged_df['embeddings'])) # similarity calculation between user's input and description

    recommended = similarities.argsort()[-top_n:][::-1] # ranks straints by similairty 

    return merged_df.iloc[recommended][['name', 'description', 'compound score']]


In [262]:
def recommend_terpene(user_input, top_n=5):
    # Generate the user embedding from the input text
    user_embedding = model.encode(user_input, convert_to_tensor=True)  # users inputs embeddings
    
    #reshaping to fix the error
    user_embedding = user_embedding.cpu().numpy() 
    if user_embedding.ndim == 1:
        user_embedding = user_embedding.reshape(1, -1) 
    # calculate the similariy between user input and embeddings 
    similarities = np.inner(user_embedding, np.vstack(merged_df['embeddings'])) 

    #rank recommendations
    recommended_indices = similarities[0].argsort()[-top_n:][::-1]

    recommendations = merged_df.iloc[recommended_indices][['name', 'description', 'compound score']]

    # Add the effects columns to the recommendations
    for effect_col in effect_columns:
        recommendations[effect_col] = merged_df.loc[recommended_indices, effect_col].values

    return recommendations

In [263]:
# gradio 
def gradio_interface(user_input):
    return recommend_terpene(user_input)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Enter a description of the desired terpene"),
    outputs=gr.Dataframe(label="Recommended Terpenes")
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.
